In [1]:
%%bash

# Dependencies
pip install docx2txt
pip install gensim
pip install keras
pip install nltk
pip install -U scikit-learn
pip install python-docx
pip install tensorflow
pip install nltk
pip install spacy
pip install csv
python3 -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
2023-05-21 04:45:48.059967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 04:45:50.462722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 04:45:50.465764: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 04:45:57.535970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-21 04:46:12.750731: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# Use if en_core_web_sm not installable via python3 in terminal:
# import spacy
# spacy.cli.download("en_core_web_sm")

In [1]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

from gensim.models import KeyedVectors

# Path to the downloaded .vec file
path_to_vectors = 'wiki-news-300d-1M.vec'
# path_to_vectors = 'wiki.en.vec'
# Load the word vectors
word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# Find similar words
similar_words = word_vectors.most_similar('cat')

# Calculate word similarity
similarity = word_vectors.similarity('cat', 'dog')

# Perform vector arithmetic
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']

In [19]:
import spacy

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [24]:
# Extract verbs and corresponding Bloom level from csv file
import csv

path_to_bloom_verbs = 'Bloom_Verbs.csv'
bloom_verbs = []

# Extract Verbs and corresponding Bloom level from csv file
import csv
with open(path_to_bloom_verbs, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] != '':
            bloom_verbs.append(row) 

bloom_levels = [
    ["Remembering"],
    ["Understanding"],
    ["Applying"],
    ["Analysing"],
    ["Evaluating"],
    ["Creating"],
]

# Store verbs into array at corresponding level
for i in range(1, len(bloom_verbs)):
    if bloom_verbs[i][1] != 0:
        match bloom_verbs[i][1]:
            case "Remembering":
                bloom_levels[0].append(str.lower(bloom_verbs[i][0]))
            case "Understanding":
                bloom_levels[1].append(str.lower(bloom_verbs[i][0]))
            case "Applying":
                bloom_levels[2].append(str.lower(bloom_verbs[i][0]))
            case "Analysing":
                bloom_levels[3].append(str.lower(bloom_verbs[i][0]))
            case "Evaluating":
                bloom_levels[4].append(str.lower(bloom_verbs[i][0]))
            case "Creating":
                bloom_levels[5].append(str.lower(bloom_verbs[i][0]))

In [25]:
# Extract LOs and corresponding mapped Bloom level from csv file
path_to_mappings = 'eng4701-mappings.csv'
sentences = []

import csv
with open(path_to_mappings, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] != '':
            sentences.append(row[0]) 

In [26]:
## Main piece of code that performs the mapping

import math
# wiki word vectors no uppercase
# TODO: Modify this section of the code to use the bloom level verbs from Arragon's spreadsheet, will also need to modify

output_bloom_levels = []

from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []

# TODO: train CLO classification with all data instead of just one course.
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    cleaned_tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token.lower() not in stop_words
    ]
    lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(
    sentences=lo_sentence_array, vector_size=100, window=5, min_count=1, workers=4, sg=1
)

# train the model with the course's ULOs and PLOs.
model.train([tokens], total_examples=len([tokens]), epochs=10)

lo_sentence_array

def bloom_mapping(sentences, final_bloom_level):
    # identified_levels = []
    final_level = None
    passed_mappings = 0
    failed_mappings = 0
    total_mappings = len(sentences)
    for i in range(1, len(sentences)): # Iterates over the LOs
        verbs = identify_verbs(sentences[i])
        # print("Verbs: ", verbs)
        score = 0
        for j in range(len(verbs)):     # Iterates over the verbs identified in each LO
            for k in range(len(bloom_levels)):  # Iterates over Bloom levels
                for l in range(1, len(bloom_levels[k])):    # Iterates over verbs in each Bloom level
                    similarity_score = -1   
                    try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
                        similarity_score = word_vectors.similarity(verbs[j], bloom_levels[k][l])
                    except:
                        pass
                    if similarity_score >= score:
                        score = similarity_score
                        final_level = bloom_levels[k][0]
                        final_level_idx = k
                        # print(bloom_levels[k][l] + " " + str(score))
        print(final_level)
        if final_bloom_level[i].lower() == final_level.lower():
            passed_mappings += 1
        else:
            failed_mappings += 1
        output_bloom_levels.append(final_level_idx)
        # print("Sentence: ", sentences[i])
        # print("Identified blooms level: ", final_level, "\n")

# print(output_bloom_levels)
    mapping_percentage = math.ceil((passed_mappings/total_mappings)*100)
    print("Total percentage of mappings passed: ", mapping_percentage, "%")
### Todos
# Find a way to use skipgrams
# This method only works for blooms since this paper is only based on blooms mapping

[nltk_data] Downloading package punkt to /home/musthafa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/musthafa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/musthafa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
import csv

def extract_columns(csv_file, columns):
    extracted_data = {}
    
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the headers
        
        # Check if all specified columns exist in the CSV file
        for column in columns:
            if column not in headers:
                raise ValueError(f"Column '{column}' not found in the CSV file.")
        
        # Initialize separate arrays for each column
        for column in columns:
            extracted_data[column] = []
        
        # Extract data from specified columns
        for row in reader:
            for column in columns:
                column_index = headers.index(column)
                extracted_data[column].append(row[column_index])
    
    return extracted_data

# Example usage
csv_file = 'Learning outcomes manual mapping - Mappings.csv'
columns_to_extract = ['Learning outcomes', 'Final Bloom Level']
extracted_data = extract_columns(csv_file, columns_to_extract)
sentences = extracted_data['Learning outcomes']
final_bloom_level = extracted_data['Final Bloom Level']
bloom_mapping(sentences, final_bloom_level)



Applying
Evaluating
Creating
Analysing
Remembering
Remembering
Evaluating
Applying
Understanding
Applying
Creating
Applying
Creating
Creating
Evaluating
Evaluating
Remembering
Understanding
Creating
Evaluating
Applying
Evaluating
Applying
Understanding
Applying
Evaluating
Evaluating
Evaluating
Analysing
Applying
Remembering
Applying
Applying
Creating
Applying
Evaluating
Applying
Applying
Understanding
Applying
Applying
Applying
Creating
Creating
Understanding
Understanding
Remembering
Applying
Applying
Understanding
Applying
Applying
Evaluating
Applying
Applying
Creating
Evaluating
Evaluating
Evaluating
Applying
Applying
Applying
Applying
Applying
Applying
Applying
Applying
Applying
Analysing
Analysing
Applying
Applying
Applying
Applying
Applying
Creating
Evaluating
Understanding
Creating
Understanding
Evaluating
Evaluating
Evaluating
Applying
Creating
Applying
Applying
Understanding
Evaluating
Evaluating
Understanding
Applying
Understanding
Remembering
Applying
Applying
Applying
Total